In [ ]:
################################### AUTO THERMAL PIPELINE (WATCHDOG + ETL) #################### 
import os
import json
import subprocess
import time
import base64
import io
import flyr
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# --- CONFIGURATION ---
INPUT_FOLDER = "flir e5 photodump"
EXIFTOOL_PATH = "exiftool-12.35.exe" 

# --- DATABASE CREDENTIALS ---
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

# ==============================================================================
# SECTION 1: DATABASE & HELPER FUNCTIONS
# ==============================================================================

def get_db_engine():
    """Creates the connection to MSSQL."""
    encoded_pass = quote_plus(DB_PASS)
    db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
    return create_engine(db_url)

def get_existing_signatures(engine, start_date_str):
    """
    SMART FILTER: Queries DB for records newer than the oldest local photo.
    """
    try:
        query = f"SELECT Asset_Name, Timestamp FROM {DB_TABLE} WHERE Timestamp >= '{start_date_str}'"
        df = pd.read_sql(query, engine)
        
        if not df.empty:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed')
            signatures = set(zip(
                df['Asset_Name'], 
                df['Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
            ))
            return signatures
        return set()
    except Exception as e:
        print(f"⚠️ Warning: Database check failed (Table might be empty): {e}")
        return set()

def get_metadata(folder):
    cmd = [
        EXIFTOOL_PATH, '-j', '-n', '-r', 
        '-DateTimeOriginal', '-CameraSerialNumber', '-ImageDescription', 
        '-Emissivity', '-ObjectDistance', '-ext', 'jpg', folder
    ]
    try:
        flags = subprocess.CREATE_NO_WINDOW if os.name == 'nt' else 0
        result = subprocess.run(cmd, capture_output=True, text=True, creationflags=flags)
        return json.loads(result.stdout)
    except Exception as e:
        print(f"Metadata scan failed: {e}")
        return []

def process_image(filepath, metadata_entry):
    filename = os.path.basename(filepath)
    asset_name = metadata_entry.get("ImageDescription")
    asset_str = str(asset_name).strip() if asset_name else ""
    
    if not asset_str: return None

    try:
        # Metadata Extraction
        serial_int = int(metadata_entry["CameraSerialNumber"])
        ts_str = str(metadata_entry["DateTimeOriginal"]).replace(":", "-", 2)
        
        # Thermal Extraction
        thermogram = flyr.unpack(filepath)
        celsius = thermogram.celsius
        
        # Stats Calculation
        h, w = celsius.shape
        cy, cx = h // 2, w // 2
        center_val = celsius[cy-1:cy+2, cx-1:cx+2].mean()
        
        row = {
            "Timestamp": ts_str, 
            "Filename": filename,
            "Camera_Serial": serial_int,
            "Asset_Name": asset_str,     
            "Max_Temp_C": round(celsius.max(), 1),
            "Min_Temp_C": round(celsius.min(), 1),
            "Avg_Temp_C": round(celsius.mean(), 1),
            "Center_Temp_C": round(center_val, 1),
            "Delta_Temp_C": round(celsius.max() - celsius.min(), 1),
            "Emissivity": float(metadata_entry.get("Emissivity", 0.95)),
            "Distance": round(float(metadata_entry.get("ObjectDistance", 1.0)), 1),
            "Image_Base64": ""
        }

        # Image Generation (JPEG)
        buffer = io.BytesIO()
        plt.imsave(buffer, celsius, cmap='inferno', format='jpeg')
        buffer.seek(0)
        raw_b64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        row["Image_Base64"] = f"data:image/jpeg;base64,{raw_b64}"
        buffer.close()
        
        return row

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return None 

# ==============================================================================
# SECTION 2: THE PIPELINE LOGIC
# ==============================================================================

def run_pipeline():
    print("\n🔄 Starting Pipeline Run...")
    
    if not os.path.exists(INPUT_FOLDER):
        print(f"❌ Error: Folder {INPUT_FOLDER} not found")
        return

    # 1. Scan Local Metadata
    meta_list = get_metadata(INPUT_FOLDER)
    if not meta_list:
        print("   No images found to process.")
        return

    meta_dict = {}
    timestamps = []
    for m in meta_list:
        if 'SourceFile' in m:
            fname = os.path.basename(m['SourceFile'])
            meta_dict[fname] = m
            if 'DateTimeOriginal' in m:
                timestamps.append(str(m['DateTimeOriginal']).replace(":", "-", 2))
    
    if not timestamps:
        print("   No valid timestamps found.")
        return

    # 2. Determine Time Window & Query DB
    oldest_photo_time = min(timestamps)
    query_start_date = oldest_photo_time[:10] + " 00:00:00"
    
    try:
        engine = get_db_engine()
        existing_signatures = get_existing_signatures(engine, query_start_date)
    except Exception as e:
        print(f"❌ DB Connection Failed: {e}")
        return

    # 3. Filter Duplicates
    files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(".jpg")]
    files_to_process = []
    
    for f in files:
        m_data = meta_dict.get(f, {})
        asset = str(m_data.get("ImageDescription", "")).strip()
        ts = str(m_data.get("DateTimeOriginal", "")).replace(":", "-", 2)[:19]
        
        if asset and ts and (asset, ts) not in existing_signatures:
            files_to_process.append(f)

    if not files_to_process:
        print("✅ No new data. All images are already in Database.")
        return

    print(f"🚀 Processing {len(files_to_process)} NEW images...")

    # 4. Process & Upload
    new_rows = []
    for f in files_to_process:
        full_path = os.path.join(INPUT_FOLDER, f)
        row = process_image(full_path, meta_dict.get(f, {}))
        if row: new_rows.append(row)

    if new_rows:
        df = pd.DataFrame(new_rows)
        # Ensure correct column order matches DB
        cols = ["Timestamp", "Filename", "Camera_Serial", "Asset_Name", 
                "Max_Temp_C", "Min_Temp_C", "Center_Temp_C", "Avg_Temp_C", 
                "Delta_Temp_C", "Emissivity", "Distance", "Image_Base64"]
        df = df[cols]
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed')
        
        try:
            df.to_sql(DB_TABLE, engine, if_exists='append', index=False)
            print(f"🎉 SUCCESS: Uploaded {len(df)} new records.")
        except Exception as e:
            print(f"❌ Upload Failed: {e}")

# ==============================================================================
# SECTION 3: WATCHDOG EVENT HANDLER
# ==============================================================================

class NewImageHandler(FileSystemEventHandler):
    def on_created(self, event):
        # Trigger when a new file is created
        if not event.is_directory and event.src_path.lower().endswith(".jpg"):
            print(f"\n👀 New file detected: {os.path.basename(event.src_path)}")
            time.sleep(1) # Wait for write to finish
            run_pipeline()

    def on_moved(self, event):
        # Trigger when a file is renamed or moved into the folder
        if not event.is_directory and event.dest_path.lower().endswith(".jpg"):
            print(f"\n👀 File move detected: {os.path.basename(event.dest_path)}")
            time.sleep(1)
            run_pipeline()

# ==============================================================================
# SECTION 4: MAIN EXECUTION
# ==============================================================================

if __name__ == "__main__":
    if not os.path.exists(INPUT_FOLDER):
        print(f"❌ Folder '{INPUT_FOLDER}' does not exist. Please create it.")
        exit()

    print(f"✅ WATCHING folder: '{INPUT_FOLDER}'")
    print("   - Drop files to trigger automatically.")
    print("   - Press [ENTER] to trigger manually.")
    print("   - Press Ctrl+C to stop.")

    # 1. Run once at startup
    run_pipeline()

    # 2. Start the Watcher in the background
    event_handler = NewImageHandler()
    observer = Observer()
    observer.schedule(event_handler, INPUT_FOLDER, recursive=False)
    observer.start()

    try:
        # CHANGED: Instead of sleeping, we wait for user input
        while True:
            input() # Waits for you to press Enter
            print("\nForce trigger requested...")
            run_pipeline()
            print("\n✅ Done. Watching...")
            
    except KeyboardInterrupt:
        observer.stop()
        print("\n🛑 Stopping Watcher.")
    
    observer.join()

Scanning folder metadata...
Detected oldest image from: 2025-12-02 00:00:00
Connecting to DB to check records since 2025-12-02 00:00:00...
Database holds 3 potential duplicates in this time range.

No new data found. (Skipped 3 duplicates).


In [16]:
#################################### POST-UPLOAD VERIFICATION & EXPORT ####################
import pandas as pd
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# --- CONFIGURATION (Must match previous cell) ---
INPUT_FOLDER = "flir e5 photodump"
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

def verify_and_export():
    print(f"🔌 Connecting to {DB_SERVER} to verify data...")
    
    # 1. Connect to MSSQL
    try:
        encoded_pass = quote_plus(DB_PASS)
        db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
        engine = create_engine(db_url)
        
        # 2. Download EVERYTHING from the Database
        print("📥 Downloading full table data...")
        query = f"SELECT * FROM {DB_TABLE} ORDER BY Timestamp DESC"
        df_full = pd.read_sql(query, engine)
        
        # Save File 1: The Full Database Dump
        df_full.to_csv("Database_Full_Dump.csv", index=False)
        print(f"✅ Saved 'Database_Full_Dump.csv' ({len(df_full)} rows)")

        # 3. Verify Local Files
        if os.path.exists(INPUT_FOLDER):
            # Get list of filenames currently in your folder
            local_files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(".jpg")]
            
            # Filter the database data to see which of your local files made it in
            # We check if the 'Filename' column in DB exists in our local file list
            df_local_verified = df_full[df_full['Filename'].isin(local_files)]
            
            # Save File 2: The Verified Uploads
            df_local_verified.to_csv("Local_Files_In_DB.csv", index=False)
            
            print(f"✅ Saved 'Local_Files_In_DB.csv' ({len(df_local_verified)} rows found matching your local folder)")
            
            # Quick Integrity Check
            missing_count = len(local_files) - len(df_local_verified)
            if missing_count == 0:
                print("🎉 SUCCESS: 100% of your local images are present in the database.")
            else:
                print(f"⚠️ WARNING: {missing_count} local images are NOT in the database yet.")
        else:
            print(f"⚠️ Could not find folder '{INPUT_FOLDER}' to verify local files.")

    except Exception as e:
        print(f"❌ Verification failed: {e}")

# Run verification
verify_and_export()

🔌 Connecting to PSQLAPPEG297-01 to verify data...
📥 Downloading full table data...
✅ Saved 'Database_Full_Dump.csv' (3 rows)
✅ Saved 'Local_Files_In_DB.csv' (3 rows found matching your local folder)
🎉 SUCCESS: 100% of your local images are present in the database.


In [21]:
##################################### DELETE ALL DATA FROM TABLE ####################
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

# --- CONFIGURATION ---
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

def delete_table_data():
    # 1. Safety Check
    confirm = input(f"⚠️ ARE YOU SURE you want to delete ALL rows from '{DB_TABLE}'? (Type 'yes' to confirm): ")
    if confirm.lower() != "yes":
        print("Action cancelled.")
        return

    try:
        print(f"Connecting to {DB_SERVER}...")
        
        # 2. Encode Password & Connect
        encoded_pass = quote_plus(DB_PASS)
        db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
        engine = create_engine(db_url)

        # 3. Execute Delete
        with engine.connect() as conn:
            # Using 'DELETE' instead of 'TRUNCATE' is safer regarding permissions
            # sql = text(f"DELETE FROM {DB_TABLE} where Filename like 'FLIR0058.jpg'")
            sql = text(f"DELETE FROM {DB_TABLE}")
            result = conn.execute(sql)
            conn.commit()
            print(f"✅ Success! Table '{DB_TABLE}' has been cleared.")
            
    except Exception as e:
        print(f"❌ Error deleting data: {e}")

# Run the delete function
delete_table_data()

Connecting to PSQLAPPEG297-01...
✅ Success! Table 'ThermalReadings' has been cleared.


In [1]:
#### QR Code ####
import os
import csv
import json
import subprocess
import io
import concurrent.futures 
import flyr
from PIL import Image
from pyzbar.pyzbar import decode

# --- CONFIGURATION ---
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    base_dir = os.getcwd()

INPUT_FOLDER = os.path.join(base_dir, "flir e5 photodump")
OUTPUT_CSV = os.path.join(base_dir, "Thermal_Data_Log.csv")
EXIFTOOL_PATH = os.path.join(base_dir, "exiftool-12.35.exe") 

# Worker count: 10 is usually safe for file operations
MAX_WORKERS = 10

# --- HELPER FUNCTIONS ---

def get_process_flags():
    if os.name == 'nt':
        return subprocess.CREATE_NO_WINDOW
    return 0

def load_existing_records(csv_path):
    existing_signatures = set()
    if not os.path.exists(csv_path):
        return existing_signatures, False 
    try:
        with open(csv_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                ts = str(row.get("Timestamp", "")).strip()
                sn = str(row.get("Serial Number", "")).strip()
                if ts and sn:
                    existing_signatures.add((ts, sn))
        return existing_signatures, True
    except Exception:
        return existing_signatures, False

def get_folder_metadata_batch(folder_path, tool_path):
    print("Scanning metadata (Batch)...")
    tool_cmd = tool_path if os.path.exists(tool_path) else "exiftool"
    try:
        cmd = [
            tool_cmd, '-j', '-n', '-r', '-ext', 'jpg', '-ext', 'jpeg',
            '-DateTimeOriginal', '-CameraModel', '-CameraSerialNumber',
            '-SerialNumber', '-Emissivity', '-ObjectDistance',
            '-ImageDescription', '-UserComment',
            folder_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, creationflags=get_process_flags())
        if result.stdout:
            return json.loads(result.stdout)
    except Exception as e:
        print(f"Error in metadata extraction: {e}")
    return []

# --- QR EXTRACTION HELPER ---
def extract_qr_with_exiftool(file_path):
    """
    Extracts the 'EmbeddedImage' (Real Photo) from the FLIR file 
    using ExifTool, then scans it for QR codes.
    """
    try:
        # Command to extract the binary image data
        cmd = [EXIFTOOL_PATH, "-b", "-EmbeddedImage", file_path]
        
        result = subprocess.run(
            cmd, 
            capture_output=True,
            creationflags=get_process_flags()
        )
        
        # If extraction worked, we have bytes
        if result.stdout:
            # Create a virtual image file in memory
            with Image.open(io.BytesIO(result.stdout)) as img:
                decoded = decode(img)
                if decoded:
                    # Return all found codes joined by pipe
                    return " | ".join([obj.data.decode("utf-8") for obj in decoded])
    except Exception:
        pass
        
    return "" # Return empty string if nothing found or error

# --- CORE PROCESSING WORKER ---
def process_single_image(meta):
    """
    Runs in a separate Thread. 
    """
    full_path = meta.get('SourceFile')
    filename = os.path.basename(full_path)
    
    # 1. Basic Metadata
    row_ts = str(meta.get("DateTimeOriginal", "Unknown"))
    row_model = str(meta.get("CameraModel", "Unknown"))
    row_sn = str(meta.get("CameraSerialNumber", ""))
    if not row_sn or row_sn == "None":
        row_sn = str(meta.get("SerialNumber", "Unknown"))
        
    # 2. Notes
    note = meta.get("ImageDescription")
    if not note:
        note = meta.get("UserComment")
    note_str = str(note).strip() if note else ""

    qr_str = ""
    status = "Success"
    
    # Thermal Data Placeholders
    center_t, max_t, min_t, avg_t, delta_t = "", "", "", "", ""

    try:
        # --- A. QR PROCESSING ---
        qr_str = extract_qr_with_exiftool(full_path)

        # --- B. THERMAL PROCESSING ---
        thermogram = flyr.unpack(full_path)
        thermal_data = thermogram.celsius
        
        min_val = thermal_data.min()
        max_val = thermal_data.max()
        avg_val = thermal_data.mean()
        delta_val = max_val - min_val
        h, w = thermal_data.shape
        center_val = thermal_data[h//2, w//2]

        center_t = f"{center_val:.1f}"
        max_t = f"{max_val:.1f}"
        min_t = f"{min_val:.1f}"
        avg_t = f"{avg_val:.1f}"
        delta_t = f"{delta_val:.1f}"

    except Exception as e:
        err_msg = str(e)
        if "not a FLIR" in err_msg or "Invalid" in err_msg:
            status = "Error: Not Radiometric"
        else:
            status = f"Error: {err_msg}"

    return [
        filename, row_ts, row_model, row_sn,
        center_t, max_t, min_t, avg_t, delta_t,
        meta.get("Emissivity", "N/A"),
        meta.get("ObjectDistance", "N/A"),
        note_str,
        qr_str,
        status
    ]

# --- MAIN EXECUTION ---
def main():
    if not os.path.exists(INPUT_FOLDER):
        print(f"Error: Input folder not found: {INPUT_FOLDER}")
        return

    # 1. Load DB
    existing_sigs, csv_exists = load_existing_records(OUTPUT_CSV)
    
    # 2. Batch Scan Metadata
    all_metadata = get_folder_metadata_batch(INPUT_FOLDER, EXIFTOOL_PATH)
    
    # 3. Filter New Files
    files_to_process = []
    for meta in all_metadata:
        if 'SourceFile' not in meta: continue
        ts = str(meta.get("DateTimeOriginal", "")).strip()
        sn = str(meta.get("CameraSerialNumber", "")).strip() or str(meta.get("SerialNumber", "")).strip()
        
        if (ts, sn) not in existing_sigs:
            files_to_process.append(meta)

    total_new = len(files_to_process)
    print(f"New images to process: {total_new}")
    if total_new == 0: return

    # 4. Threaded Processing
    headers = [
        "File Name", "Timestamp", "Camera Model", "Serial Number", 
        "Center Temp (C)", "Max Temp (C)", "Min Temp (C)", 
        "Avg Temp (C)", "Delta Temp (C)", 
        "Emissivity", "Distance", "Notes", "QR Data", "Status"
    ]

    with open(OUTPUT_CSV, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if not csv_exists:
            writer.writerow(headers)

        print(f"Starting parallel processing with {MAX_WORKERS} threads...\n")
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            future_to_meta = {executor.submit(process_single_image, meta): meta for meta in files_to_process}
            
            for i, future in enumerate(concurrent.futures.as_completed(future_to_meta), 1):
                try:
                    row = future.result()
                    writer.writerow(row)
                    f.flush() 
                    
                    # --- UPDATED PROGRESS LINE ---
                    # end='\r' returns cursor to start of line
                    # " "*20 adds blank space to clear any previous longer text
                    filename = row[0]
                    print(f"Progress [{i}/{total_new}]: {filename}" + " "*30, end='\r', flush=True)
                    
                except Exception as exc:
                    print(f"\nError processing file: {exc}")

    print(f"\nDone! Processed {total_new} files.")

if __name__ == "__main__":
    main()

Scanning metadata (Batch)...
New images to process: 0
